In [9]:
using LinearAlgebra
using SparseArrays
using Plots

In [13]:
function xy(
    x0::Float64, 
    y0::Float64, 
    L1::Float64, 
    L2::Float64, 
    noelms1::Int64, 
    noelms2::Int64
)::Tuple{Vector{Float64}, Vector{Float64}}
    VX = repeat(collect(LinRange(x0, x0+L1, noelms1+1)), inner=noelms2+1)
    VY = repeat(collect(LinRange(y0+L2, y0, noelms2+1)), noelms1+1)
    return VX, VY
end

function conelmtab(noelms1::Int64, noelms2::Int64)::Matrix{Int64}
    k = [i for i in 1:(noelms1*(noelms2+1)) if i % (noelms2+1) != 0]

    return [
        k (2 + noelms2 .+ k) (1 + noelms2 .+ k);
        k (1 .+ k) (2 + noelms2 .+ k)
    ]
end

function basfun(
    VX::Vector{Float64}, 
    VY::Vector{Float64}, 
    EToV::Matrix{Int64}
)::Tuple{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}}
    xjs = VX[EToV[:, [2,3,1]]]
    yjs = VY[EToV[:, [2,3,1]]]

    xks = VX[EToV[:, [3,1,2]]]
    yks = VY[EToV[:, [3,1,2]]]

    as = xjs .* yks - xks .* yjs
    bs = yjs - yks
    cs = xks - xjs

    return as, bs, cs
end

basfun (generic function with 2 methods)

# Exercise 5.1

In [32]:
function assembly(
    VX::Vector{Float64},
    VY::Vector{Float64},
    EToV::Matrix{Int64},
    lam1::Float64,
    lam2::Float64,
    qt::Vector{Float64},
    c::Vector{Float64}
)::Tuple{SparseMatrixCSC{Float64, Int64}, SparseMatrixCSC{Float64, Int64}, Vector{Float64}}
    N = size(EToV)[1]
    M = length(VX)

    R = spzeros(M, M)
    S = spzeros(M, M)
    b = zeros(M)

    ats, bts, cts = basfun(VX, VY, EToV)
    deltas = sum(ats, dims=2) ./ 2
    qs = abs.(deltas) .* sum(qt[EToV], dims=2) / 9
    cs = sum(c[EToV], dims=2) / 3
    c_matrix = Diagonal(fill(1/12, 3)) + fill(1/12,3,3) # specific heat matrix for computation

    for n in 1:N
        delta = abs(deltas[n])
        cn_matrix = c_matrix .* delta .* cs[n]
        q = qs[n]
        bt = bts[n, :]
        ct = cts[n, :]

        for r in 1:3
            i = EToV[n,r]
            b[i] += q

            for s in r:3
                j = EToV[n,s]
                kn = (lam1*bt[r]*bt[s] + lam2*ct[r]*ct[s]) / (4 * delta)
                R[min(i, j), max(i, j)] += kn
                S[min(i, j), max(i ,j)] += cn_matrix[r,s]

            end
        end
    end

    return R, S, b
end

assembly (generic function with 1 method)

In [33]:
ct(x,y) = 1.0
qt(x,y) = 0.0
lam1 = lam2 = nu = 2.0
noelms1 = 4
noelms2 = 3
x0 = 0.0
y0 = 0.0
L1 = 1.0
L2 = 1.0

VX, VY = xy(x0, y0, L1, L2, noelms1, noelms2)
EToV = conelmtab(noelms1, noelms2)

r, s, b = assembly(VX, VY, EToV, lam1, lam2, qt.(VX, VY), ct.(VX,VY))
display(spy(r,markersize=5))
display(spy(s,markersize=5))
display(b)

LoadError: UndefVarError: `shn_matrix` not defined